In [2]:
import sys
if '/Users/brandon/Documents/Repositories/Python/FANC_auto_recon/transforms/' not in sys.path:
    sys.path.append('/Users/brandon/Documents/Repositories/Python/FANC_auto_recon/transforms/')
from fanc_seg_utils.database import Neuron_database
import numpy as np
from matplotlib import pyplot as plt
import pymaid
from scipy.spatial import distance 
from concurrent import futures
from matplotlib import pyplot as plt
from scipy import stats
from scipy.stats import norm
import pandas as pd
import marshmallow as mm
from cloudvolume import CloudVolume
from fanc_seg_utils import neuroglancer_utilities
import collections
import six
import json
import meshparty
from meshparty import trimesh_io, trimesh_vtk


In [17]:
db = Neuron_database('/Users/brandon/Documents/Repositories/Python/Neuron_Database/10_B.csv',segmentation_version='V4')
db.get_database()

Database active


In [18]:
db.neurons

,Segment_ID,Name,V4_Soma,V3_Soma,Annotations,Extra_Segment_IDs,Catmaid_Skids
0,73116768675457228,10b_P1_T1R,"[33263, 92148, 2923]","[33257.75, 92156.5, 2923.0]","[10b, posterior_only, right_soma, good]",NaN,{'60': 1096782}
1,72905731296619635,10b_AP1_T1R,"[34909, 93172, 2961]","[34884.0, 93203.0, 2961.0]","[10b, ap_projection, right_soma, major_split]",NaN,
2,73186450358851767,10b_P2_T1R,"[32720, 89840, 2833]","[32686.0, 89826.25, 2833.0]","[10b, posterior_only, right_soma, major_split]","[73540355328620489, 73469574469027893]",
3,73116219053837594,10b_P3_T1R,"[32201, 92967, 2910]","[32200.5, 92934.0, 2910.0]","[10b, posterior_only, right_soma, good]",NaN,{'60': 1096794}
4,73116150401377743,10b_P4_T1R,"[32549, 91784, 2972]","[32517.75, 91796.0, 2972.0]","[10b, posterior_only, right_soma, good]",[72976237278295055],{'60': 1096807}
5,73468818554780401,10b_AP2_T1R,"[31537, 96177, 2937]","[31505.5, 96148.75, 2937.0]","[10b, right_soma, good, ap_axon_class, A_T1_02]",[73467031177206040],{'60': 1096820}
6,73116150401393317,10b_AP3_T1R,"[31067, 91447, 3005]","[31034.0, 91388.75, 3005.0]","[10b, right_soma, good, ap_axon_class]","[73399205657671336, 73398655968842904]",{'60': 1096836}
7,73257025194431793,10b_AP4_T1R,"[34785, 94638, 2623]","[34784.5, 94733.0, 2623.0]","[10b, right_soma, glial_merge, ap_axon_class]","[73469437097025033, 73609281500719038, 7318672...",
8,73116768809466559,10b_P5_T1R,"[36991, 93700, 2832]","[37030.0, 93718.5, 2832.0]","[10b, posterior_only, right_soma, good]",[73398999700388486],{'60': 1096849}
9,73187137486815168,10b_P6_T1R,"[33865, 93567, 2945]","[33860.25, 93575.5, 2945.0]","[10b, posterior_only, right_soma, good]",[73116700023095946],{'60': 1096862}


In [3]:
cv = CloudVolume

Shard Indices: 1it [00:00,  2.27it/s]
Minishard Indices: 1it [00:00,  2.51it/s]
Decompressing: 100%|██████████| 1/1 [00:00<00:00, 110.30it/s]


In [6]:
sk

,neuron_name,skeleton_id,n_nodes,n_connectors,n_branch_nodes,n_end_nodes,open_ends,cable_length,review_status,soma
0,MN_A101_T1R,718764825,7109,0,95,2275,0,1916.352322,NA,True


In [ ]:
mesh = db.get_mesh(73820112385702160)
sk = db.get_skeletons(73820112385702160)

{'V4': 'https://storage.googleapis.com/zetta_lee_fly_vnc_001_segmentation/vnc1_full_v3align_2/realigned_v1/seg/full_run_v1',
 'V4_2': 'https://storage.googleapis.com/zetta_lee_fly_vnc_001_segmentation/vnc1_full_v3align_2/realigned_v1/seg/full_run_v2',
 'V4_dynamic': 'graphene://https://standalone.poyntr.co/segmentation/table/vnc1_full_v3align_2'}

In [16]:
cv = CloudVolume('graphene://https://api.zetta.ai/wclee/segmentation/table/vnc1_full_v3align_2_v2')

UnsupportedFormatError: Unable to parse Graphene URL: https://api.zetta.ai/wclee/segmentation/table/vnc1_full_v3align_2_v2

In [ ]:
db.initialize_catmaid_instance('/Users/brandon/Documents/MN_Analysis/catmaid_keys.txt',60)

In [ ]:
output = neuroglancer_utilities.fanc4_to_3(nodes.values)
neuron.nodes[['x','y','z']] = np.array(list(zip(output['x'],output['y'],output['z']))) * np.array([4.3,4.3,45])

In [21]:
CloudVolume('graphene://https://wclee.dev.zetta.ai/segmentation/table/vnc1_full_v3align_2_v2', use_https=True, agglomerate=True)

HTTPError: 401 Client Error: Unauthorized for url: https://wclee.dev.zetta.ai/segmentation/table/vnc1_full_v3align_2_v2/info

In [8]:
1916/2889.352

0.6631244652780278

In [ ]:
def skeletor_contraction(a,**bc):   
    cmesh = add(a,**bc)
    return(cmesh)

def add(a,b,c):
    print(a+b+c)

In [ ]:
skeletor_contraction(a,**bc)

In [ ]:
db.upload_to_catmaid(73469574737466912,only_good=False)

In [ ]:
seg_id = 648518346661938249
sk = single_skeletonization.single_skeleton(seg_id,
                                       cv_url=db.segmentations[db.segmentation_version],
                                       soma_coords=db.neurons.loc[db.neurons.Segment_ID == seg_id,'V4_Soma'])

In [ ]:

class NumericField(mm.fields.Int):

    def _jsonschema_type_mapping(self):
        return {
            'type': 'integer',
        }
 
class AnnotationSchema(mm.Schema):
    '''schema with the type of annotation'''
    type = mm.fields.Str(
        required=True,
        description='type of annotation',
        drop_column=True)

    valid = mm.fields.Bool(
        required=False,
        description="is this annotation valid"
    )
    
    
class SpatialPoint(mm.Schema):
    '''a position in the segmented volume '''
    position = mm.fields.List(mm.fields.Int,
                              validate=mm.validate.Length(equal=3),
                              required=True,
                              description='spatial position in voxels of'
                                          'x,y,z of annotation',
                              postgis_geometry='POINTZ')

    @mm.post_load
    def transform_position(self, item):
        if self.context.get('postgis', False):
            item['position'] = "POINTZ({} {} {})".format(item['position'][0],
                                                         item['position'][1],
                                                         item['position'][2])
        return item


class BoundSpatialPoint(SpatialPoint):
    ''' a position in the segmented volume that is associated with an object'''
    supervoxel_id = NumericField(missing=mm.missing,
                                 description="supervoxel id of this point")
    root_id = NumericField(description="root id of the bound point",
                           index=True)

    @mm.post_load
    def convert_point(self, item):
        bsp_fn = self.context.get('bsp_fn', None)
        if bsp_fn is not None:
            bsp_fn(item)
        return item

In [ ]:
a = SpatialPoint.from_dict({'postition':[41854, 153786, 2396]})

In [ ]:
a.postition

In [ ]:
db.segmentations

In [ ]:
db.neurons.loc[['good' in i for i in db.neurons.Annotations],'Segment_ID'].values

In [ ]:
db.plot_mesh(74172093612626737)

In [ ]:
volume = db.get_mesh(74172093612626737)

In [ ]:
volume

In [ ]:
pvol = pymaid.Volume(volume[0].vertices,volume[0].faces,name=db.neurons.loc[db.neurons.Segment_ID==74172093612626737,'Name'])

In [ ]:
db.initialize_catmaid_instance('/Users/brandon/Documents/MN_Analysis/catmaid_keys.txt',60)